# Titulo

- Ana Margarida Ruivo Loureiro - up201705749

- João Martins

- João Ricardo Cardoso

## Abstract

## Introdução

## Dataset

O nosso dataset providencia-nos dados geográficos e contagem de casos confirmados, óbitos e recuperados do Covid-19 num periodo de  4 meses (22 de janeiro de 2020 a 5 de maio de 2020) em vários países. O objetivo será extrair informação dos dados de forma a podermos construir um modelo de regressão capaz de prever as contagens tendo por base os fatores disponiveis (localização geográfica, dia e contagem anterior). Assim sendo é necessário partir dos dados iniciais e proceder a um pré-processamento para tratar os dados e poder-se então utlizar os algoritmos de regressão que pretendemos utilizar.

### Variáveis independentes

Como foi supramencionado, pretendemos utilizar certos fatores para fundamentarmos a nossa previsão (partindo dos dados do dataset). Estes então serão as coordenadas geográficas (Latitude e Longitude), número de dias desde o inicio da previsão (ou seja, 22 de janeiro) e contagens anteriores (número de casos confirmados, óbitos e recuperados do dia anterior).

### Variáveis Dependentes

Os valores que vamos prever serão os números de casos confirmados, óbitos e recuperados, sendo então estas as nossas variáveis independentes.

## Tratamento de dados

Para podermos obter resultados fiáveis, é preciso haver um tratamento dos dados que vem do dataset.
Pegando dos dados processados anteriormente feito inicialmente, começamos por extrai-los do ficheiro para podermos manipulá-los.

In [2]:
import pandas as pd

covid_data = pd.read_csv('covid_19_clean_complete.csv')
covid_data

,Province/State,Country/Region,Lat,Long,Date,Confirmed,Deaths,Recovered
0,NaN,Afghanistan,33.000000,65.000000,1/22/20,0,0,0
1,NaN,Albania,41.153300,20.168300,1/22/20,0,0,0
2,NaN,Algeria,28.033900,1.659600,1/22/20,0,0,0
3,NaN,Andorra,42.506300,1.521800,1/22/20,0,0,0
4,NaN,Angola,-11.202700,17.873900,1/22/20,0,0,0
...,...,...,...,...,...,...,...,...
27451,NaN,Western Sahara,24.215500,-12.885800,5/4/20,6,0,5
27452,NaN,Sao Tome and Principe,0.186360,6.613081,5/4/20,23,3,4
27453,NaN,Yemen,15.552727,48.516388,5/4/20,12,2,0
27454,NaN,Comoros,-11.645500,43.333300,5/4/20,3,0,0


De seguida, após uma análise, verificou-se a presença de entrada respeitantes a navios que nalgum momento tiveram casos de Covid-19 e não estão portanto associados a nenhum país particular.
Além disso, consideramos que estes dados iriam criar ruído, pelo que optamos por ignorá-los e remover dos dados em análise.

In [3]:
covid_data = covid_data.drop(covid_data[covid_data['Province/State']=='Grand Princess'].index)
covid_data = covid_data.drop(covid_data[covid_data['Province/State']=='Diamond Princess'].index)
covid_data = covid_data.drop(covid_data[covid_data['Country/Region']=='Diamond Princess'].index)
covid_data = covid_data.drop(covid_data[covid_data['Country/Region']=='MS Zaandam'].index)
covid_data = covid_data.reset_index()
del covid_data['index']
covid_data

,Province/State,Country/Region,Lat,Long,Date,Confirmed,Deaths,Recovered
0,NaN,Afghanistan,33.000000,65.000000,1/22/20,0,0,0
1,NaN,Albania,41.153300,20.168300,1/22/20,0,0,0
2,NaN,Algeria,28.033900,1.659600,1/22/20,0,0,0
3,NaN,Andorra,42.506300,1.521800,1/22/20,0,0,0
4,NaN,Angola,-11.202700,17.873900,1/22/20,0,0,0
...,...,...,...,...,...,...,...,...
27035,NaN,Western Sahara,24.215500,-12.885800,5/4/20,6,0,5
27036,NaN,Sao Tome and Principe,0.186360,6.613081,5/4/20,23,3,4
27037,NaN,Yemen,15.552727,48.516388,5/4/20,12,2,0
27038,NaN,Comoros,-11.645500,43.333300,5/4/20,3,0,0


A maioria das entradas da coluna **Province/State** tem valores nulos, pelo que procedemos a eliminá-los. Além disso, como ter uma entrada para uma região e país não é muito relevante, optamos por agregar as duas informações numa só coluna denominada de **Local**.

In [4]:
import numpy as np

covid_data['Province/State'] = covid_data.replace(np.nan, '', regex=True)
cols = ['Province/State', 'Country/Region']
covid_data['Local'] = covid_data[cols].apply(lambda row: ' / '.join(row.values.astype(str)) if row.values[0] != '' else ''.join(row.values.astype(str)), axis=1)
del covid_data['Province/State']
del covid_data['Country/Region']
covid_data

,Lat,Long,Date,Confirmed,Deaths,Recovered,Local
0,33.000000,65.000000,1/22/20,0,0,0,Afghanistan
1,41.153300,20.168300,1/22/20,0,0,0,Albania
2,28.033900,1.659600,1/22/20,0,0,0,Algeria
3,42.506300,1.521800,1/22/20,0,0,0,Andorra
4,-11.202700,17.873900,1/22/20,0,0,0,Angola
...,...,...,...,...,...,...,...
27035,24.215500,-12.885800,5/4/20,6,0,5,Western Sahara
27036,0.186360,6.613081,5/4/20,23,3,4,Sao Tome and Principe
27037,15.552727,48.516388,5/4/20,12,2,0,Yemen
27038,-11.645500,43.333300,5/4/20,3,0,0,Comoros


De seguida, vamos converter as datas em contagem de dias desde o início do dataset (22 de janeiro de 2020)

In [5]:
covid_data['Date'] = pd.to_datetime(covid_data['Date'],format='%m/%d/%y')
covid_data['Date'] -= pd.to_datetime("2020-01-22")
covid_data['Date'] /= np.timedelta64(1,'D')
covid_data = covid_data.rename(columns  = {'Date':'Days Passed'})
covid_data

,Lat,Long,Days Passed,Confirmed,Deaths,Recovered,Local
0,33.000000,65.000000,0.0,0,0,0,Afghanistan
1,41.153300,20.168300,0.0,0,0,0,Albania
2,28.033900,1.659600,0.0,0,0,0,Algeria
3,42.506300,1.521800,0.0,0,0,0,Andorra
4,-11.202700,17.873900,0.0,0,0,0,Angola
...,...,...,...,...,...,...,...
27035,24.215500,-12.885800,103.0,6,0,5,Western Sahara
27036,0.186360,6.613081,103.0,23,3,4,Sao Tome and Principe
27037,15.552727,48.516388,103.0,12,2,0,Yemen
27038,-11.645500,43.333300,103.0,3,0,0,Comoros


Por fim, adicionar as colunas da contagem do dia anterior. Este passo é um pouco mais longo tendo em conta o numero de linhas e a procura pelo valor anterior.

In [6]:
covid_data['Conf. Prev.'] = covid_data.apply(lambda row: 
                                                      covid_data[(covid_data['Local'] == row['Local']) & (covid_data['Days Passed'] == row['Days Passed']-1)]['Confirmed'].item()
                                                      if row['Days Passed'] > 0 else 0,axis=1)
covid_data['Deaths Prev.'] = covid_data.apply(lambda row: 
                                                      covid_data[(covid_data['Local'] == row['Local']) & (covid_data['Days Passed'] == row['Days Passed']-1)]['Deaths'].item()
                                                      if row['Days Passed'] > 0 else 0,axis=1)
covid_data['Recov. Prev.'] = covid_data.apply(lambda row: 
                                                      covid_data[(covid_data['Local'] == row['Local']) & (covid_data['Days Passed'] == row['Days Passed']-1)]['Recovered'].item()
                                                      if row['Days Passed'] > 0 else 0,axis=1)
covid_data

,Lat,Long,Days Passed,Confirmed,Deaths,Recovered,Local,Conf. Prev.,Deaths Prev.,Recov. Prev.
0,33.000000,65.000000,0.0,0,0,0,Afghanistan,0,0,0
1,41.153300,20.168300,0.0,0,0,0,Albania,0,0,0
2,28.033900,1.659600,0.0,0,0,0,Algeria,0,0,0
3,42.506300,1.521800,0.0,0,0,0,Andorra,0,0,0
4,-11.202700,17.873900,0.0,0,0,0,Angola,0,0,0
...,...,...,...,...,...,...,...,...,...,...
27035,24.215500,-12.885800,103.0,6,0,5,Western Sahara,6,0,5
27036,0.186360,6.613081,103.0,23,3,4,Sao Tome and Principe,16,1,4
27037,15.552727,48.516388,103.0,12,2,0,Yemen,10,2,0
27038,-11.645500,43.333300,103.0,3,0,0,Comoros,3,0,0


In [7]:
with pd.ExcelWriter('covid_19_distance.xlsx') as writer:
    covid_data.to_excel(writer)

# Solução

## K Nearest Neighbours
Este dataset que processamos permite-nos agrupar certos fatores relevantes como a posição geográfica e contagem dos dias anteriores para  poder-se relacionar certos pontos (entradas de dados) e obter uma previsão dos dados tendo em conta aquelas que estão mais próximas, que é em que se baseia o algoritmo **K Nearest neighbours**.
Já que pretendemos construir um modelo de regressão destes dados, vamos recorrer à ferramenta *scikit-learn* para utilizar os seus algoritmos de regressão, nos quais se encontra o **KNeighborsRegressor**, que será o escolhido para aplicar o KNN.

### KNeighborsRegressor
#### Parâmetros da pesquisa
* **n_neighbors**: número de vizinhos para usar, por defeito, 5.
* **weights**: função de peso utilizado na previsão. Valores a testar:
    * *uniform*: pesos uniformes, todos os pontos na vizinhança são pesados igualmente
    * *distance*: pesos influenciados pela distância ao ponto de pesquisa, em que pontos vizinhos mais pertos do ponto de pesquisa terão mais influência do que aqueles mais afastados
* **algorithm**: algoritmo usado para computar os nearest neighbors. Opção a usar:
    * *auto*: tenta decidir qual a melhor escolha, tendo em conta os valores passados para a função de **fit()** dentro das possíveis escolhas, *ball_tree*, *kd_tree* ou *brute*
* **n_jobs**: definir nº de processos para paralelizar os trabalhos
    * *None*: não há paralelismo (usar em debug)
    * *-1*: todos os cpu's são usados
    
#### Criação dos sets de treino e teste
Procedemos então à criação de sets para podermos treinar o nosso modelo, e por fim testá-lo.

In [21]:
from sklearn.model_selection import train_test_split


#criar set de treino e teste
train, test = train_test_split(covid_data, test_size=0.0096)
train

,Lat,Long,Days Passed,Confirmed,Deaths,Recovered,Local,Conf. Prev.,Deaths Prev.,Recov. Prev.
20139,7.9465,-1.0232,77.0,313,6,34,Ghana,287,5,31
13479,54.2361,-4.5481,51.0,0,0,0,Isle of Man / United Kingdom,0,0,0
10957,53.7609,-98.8139,42.0,0,0,0,Manitoba / Canada,0,0,0
20137,42.3154,43.3569,77.0,211,3,50,Georgia,196,3,46
24233,23.3417,113.4244,93.0,1585,8,1534,Guangdong / China,1585,8,1528
...,...,...,...,...,...,...,...,...,...,...
15634,3.8480,11.5021,60.0,40,0,0,Cameroon,27,0,0
18040,58.5953,25.0136,69.0,745,4,26,Estonia,715,3,20
20524,21.9162,95.9560,78.0,23,3,2,Burma,22,3,0
15509,12.8654,-85.2072,59.0,2,0,0,Nicaragua,1,0,0


In [22]:
test

,Lat,Long,Days Passed,Confirmed,Deaths,Recovered,Local,Conf. Prev.,Deaths Prev.,Recov. Prev.
24313,53.1424,-7.6921,93.0,18184,1014,9233,Ireland,17607,794,9233
6375,43.0000,12.0000,24.0,3,0,0,Italy,3,0,0
13266,-38.4161,-63.6167,51.0,31,2,0,Argentina,19,1,0
4830,-18.7669,46.8691,18.0,0,0,0,Madagascar,0,0,0
4594,21.0000,57.0000,17.0,0,0,0,Oman,0,0,0
...,...,...,...,...,...,...,...,...,...,...
11281,-26.5225,31.4659,43.0,0,0,0,Eswatini,0,0,0
5845,41.9029,12.4534,22.0,0,0,0,Holy See,0,0,0
23294,21.0079,10.9408,89.0,7,1,6,Mauritania,7,1,6
16916,47.5162,14.5501,65.0,7657,58,225,Austria,6909,49,112


In [23]:
#colunas em que vamos basear as previsões
x_columns = ['Lat','Long','Days Passed', 'Conf. Prev.','Deaths Prev.','Recov. Prev.']
#colunas que queremos prever
y_columns = ['Confirmed','Deaths','Recovered']

from sklearn.neighbors import KNeighborsRegressor
#criar o modelo e usar o número de vizinhos default, 5.
knn = KNeighborsRegressor(n_neighbors=5)
#aplicar a função de fit ao set de treino
knn.fit(train[x_columns],train[y_columns])
#fazer previsões do set de teste usando 
predictions = knn.predict(test[x_columns])

#tratar dos resultados
predictions = pd.DataFrame(data=predictions,columns=['Confirmed Prediction','Deaths Prediction','Recovered Prediction'])
predictions['Local'] = test['Local'].tolist()
#arredondar os valores para inteiros, com teto
predictions['Confirmed Prediction'] = predictions['Confirmed Prediction'].apply(np.ceil)
predictions['Deaths Prediction'] = predictions['Deaths Prediction'].apply(np.ceil)
predictions['Recovered Prediction'] = predictions['Recovered Prediction'].apply(np.ceil)
predictions['Confirmed Actual'] = test['Confirmed'].tolist()
predictions['Deaths Actual'] = test['Deaths'].tolist()
predictions['Recovered Actual'] = test['Recovered'].tolist()
predictions['Days Passed'] = test['Days Passed'].tolist()
predictions = predictions[['Days Passed','Local', 'Confirmed Prediction', 'Confirmed Actual','Deaths Prediction', 'Deaths Actual','Recovered Prediction','Recovered Actual']]
predictions

,Days Passed,Local,Confirmed Prediction,Confirmed Actual,Deaths Prediction,Deaths Actual,Recovered Prediction,Recovered Actual
0,93.0,Ireland,18706.0,18184,691.0,1014,9463.0,9233
1,24.0,Italy,3.0,3,0.0,0,0.0,0
2,51.0,Argentina,22.0,31,1.0,2,0.0,0
3,18.0,Madagascar,0.0,0,0.0,0,0.0,0
4,17.0,Oman,0.0,0,0.0,0,0.0,0
...,...,...,...,...,...,...,...,...
255,43.0,Eswatini,0.0,0,0.0,0,0.0,0
256,22.0,Holy See,0.0,0,0.0,0,0.0,0
257,89.0,Mauritania,7.0,7,1.0,1,6.0,6
258,65.0,Austria,7080.0,7657,149.0,58,43.0,225


In [10]:
covid_data['Date']

KeyError: 'Date'

## SVR

**Support Vector Regression** utiliza conceitos semelhantes aos aplicados no algoritmo de Support Vector Machine para aplicações em métodos de regressão.

Deste modo serão expostos alguns conceitos teóricos fundamentais para que se possa compreender este algoritmo:
-  **Hyper Plane** : Uma linha de separação que irá ajudar a prever os valores em causa.
-   **Boundary line** : Margens da hyperplane que separam os valores existentes.
-  **Support vectors** : Os pontos mais perto da "boundary line". A distância dos pontos é a minima.
    
Assim, neste algoritmo o objetivo é considerar os valores dentro da margem, ou seja aqueles com menor erro entre eles. Assim o objetivo é descobrir um hyper plain que mais se aproxime aos valores existentes, ou seja, que se tenha uma distância minima a um maior número destes pontos.

Desta forma a aplicação deste algoritmo vai ser feita auxiliando-nos da implementação presente na ferramenta scikit-learn. 

### Support Vector Regression

#### Parâmetros da pesquisa

Serão refer

   * **kernel** : rbf, sigmod foi implementado o default uma vez que a obtenção de resultados era muito superior às restantes opções
   * **epsilonfloat**: 0.1 e 0.2 - o valor associado ao valor da distância entre a "boundary line" e o "hyper plane"
   * **cache size**: 500 - permite diminuir o tempo de execução 
   * **C** : 1 uma vez que os dados com que trabalhamos não tem muito ruído

In [8]:
from sklearn.svm import SVR

In [9]:
covid_data

,Lat,Long,Days Passed,Confirmed,Deaths,Recovered,Local,Conf. Prev.,Deaths Prev.,Recov. Prev.
0,33.000000,65.000000,0.0,0,0,0,Afghanistan,0,0,0
1,41.153300,20.168300,0.0,0,0,0,Albania,0,0,0
2,28.033900,1.659600,0.0,0,0,0,Algeria,0,0,0
3,42.506300,1.521800,0.0,0,0,0,Andorra,0,0,0
4,-11.202700,17.873900,0.0,0,0,0,Angola,0,0,0
...,...,...,...,...,...,...,...,...,...,...
27035,24.215500,-12.885800,103.0,6,0,5,Western Sahara,6,0,5
27036,0.186360,6.613081,103.0,23,3,4,Sao Tome and Principe,16,1,4
27037,15.552727,48.516388,103.0,12,2,0,Yemen,10,2,0
27038,-11.645500,43.333300,103.0,3,0,0,Comoros,3,0,0


De forma semelhante ao algoritmo anterior procedemos à criação de sets para podermos treinar o nosso modelo, e por fim testá-lo.

In [10]:
from sklearn.model_selection import train_test_split


#criar set de treino e teste
train, test = train_test_split(covid_data, test_size=0.0096)
train

,Lat,Long,Days Passed,Confirmed,Deaths,Recovered,Local,Conf. Prev.,Deaths Prev.,Recov. Prev.
8414,18.735700,-70.162700,32.0,0,0,0,Dominican Republic,0,0,0
14062,53.709800,27.953400,54.0,36,0,3,Belarus,27,0,3
26057,39.549000,116.130600,100.0,328,6,318,Hebei / China,328,6,318
10259,7.946500,-1.023200,39.0,0,0,0,Ghana,0,0,0
17595,30.375300,69.345100,67.0,1597,14,29,Pakistan,1495,12,29
...,...,...,...,...,...,...,...,...,...,...
448,12.984300,-61.287200,1.0,0,0,0,Saint Vincent and the Grenadines,0,0,0
13961,51.919400,19.145100,53.0,119,3,0,Poland,103,3,0
19237,15.552727,48.516388,73.0,0,0,0,Yemen,0,0,0
13711,14.497400,-14.452400,52.0,10,0,1,Senegal,10,0,1


In [11]:
#colunas em que vamos basear as previsões
x_columns = ['Lat','Long','Days Passed', 'Conf. Prev.','Deaths Prev.','Recov. Prev.']
#colunas que queremos prever
y_columns = ['Confirmed','Deaths','Recovered']

O uso de apenas este algoritmo não possibilita a previsão para mais do que um output.
Assim, foi preciso auxiliarmo-nos num "wrapper" - MultiOutputRegressor - de forma a conseguirmos
contornar este problema. Esta classe irá criar uma instância do modelo para cada um dos outputs do problema.

Contudo como referido, é criado um modelo separado para cada output. Consequentemente, esta
solução não consegue garantir a dependência entre as várias varíáveis, ou seja, assume que
os outputs são totalmente independentes uns dos outros. 

No contexto deste problema os outputs tem relação entre si uma vez que o maior número de casos confirmados 
implicam um maior número de mortes e recuperados. Apesar disto, decidimos avançar na mesma com este algoritmo uma
vez que a dependência entre os vários outputs poderá não ser assim tão significativa para os resultados finais.


In [12]:
from sklearn.multioutput import MultiOutputRegressor

In [13]:
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler

## RBF with epsilon=0.1

In [14]:
regressor_rbf = make_pipeline(StandardScaler(),SVR(kernel='rbf', cache_size=500))
wrapper_rbf = MultiOutputRegressor(regressor_rbf)

In [15]:
wrapper_rbf.fit(train[x_columns],train[y_columns])
predictions_rbf = wrapper_rbf.predict(test[x_columns])

In [16]:
predictions_rbf = pd.DataFrame(data=predictions_rbf,columns=['Confirmed Prediction','Deaths Prediction','Recovered Prediction'])
predictions_rbf['Local'] = test['Local'].tolist()
predictions_rbf['Confirmed Prediction'] = predictions_rbf['Confirmed Prediction'].apply(np.ceil)
predictions_rbf['Deaths Prediction'] = predictions_rbf['Deaths Prediction'].apply(np.ceil)
predictions_rbf['Recovered Prediction'] = predictions_rbf['Recovered Prediction'].apply(np.ceil)
predictions_rbf['Confirmed Actual'] = test['Confirmed'].tolist()
predictions_rbf['Deaths Actual'] = test['Deaths'].tolist()
predictions_rbf['Recovered Actual'] = test['Recovered'].tolist()
predictions_rbf['Days Passed'] = test['Days Passed'].tolist()
predictions_rbf = predictions_rbf[['Days Passed','Local', 'Confirmed Prediction', 'Confirmed Actual','Deaths Prediction', 'Deaths Actual','Recovered Prediction','Recovered Actual']]
predictions_rbf

,Days Passed,Local,Confirmed Prediction,Confirmed Actual,Deaths Prediction,Deaths Actual,Recovered Prediction,Recovered Actual
0,13.0,Nepal,13.0,1,-0.0,0,1.0,0
1,2.0,Queensland / Australia,6.0,0,2.0,0,4.0,0
2,44.0,Sao Tome and Principe,-5.0,0,-0.0,0,-2.0,0
3,35.0,Iraq,29.0,5,-0.0,0,3.0,0
4,60.0,Faroe Islands / Denmark,188.0,115,5.0,0,13.0,0
...,...,...,...,...,...,...,...,...
255,96.0,Togo,175.0,98,5.0,6,44.0,62
256,11.0,Lithuania,-2.0,0,-0.0,0,-0.0,0
257,43.0,Cambodia,52.0,1,1.0,0,16.0,1
258,19.0,Cote d'Ivoire,-4.0,0,-0.0,0,1.0,0


In [17]:
with pd.ExcelWriter('results_svr_rbf.xlsx') as writer:
    covid_data.to_excel(writer)

## Sigmoid with epsilon=0.1

In [18]:
regressor_s = make_pipeline(StandardScaler(), SVR(kernel='sigmoid', cache_size=500))
wrapper_s = MultiOutputRegressor(regressor_s)

In [19]:
wrapper_s.fit(train[x_columns],train[y_columns])
predictions_s = wrapper_s.predict(test[x_columns])

In [20]:
predictions_s = pd.DataFrame(data=predictions_s,columns=['Confirmed Prediction','Deaths Prediction','Recovered Prediction'])
predictions_s['Local'] = test['Local'].tolist()
predictions_s['Confirmed Prediction'] = predictions_s['Confirmed Prediction'].apply(np.ceil)
predictions_s['Deaths Prediction'] = predictions_s['Deaths Prediction'].apply(np.ceil)
predictions_s['Recovered Prediction'] = predictions_s['Recovered Prediction'].apply(np.ceil)
predictions_s['Confirmed Actual'] = test['Confirmed'].tolist()
predictions_s['Deaths Actual'] = test['Deaths'].tolist()
predictions_s['Recovered Actual'] = test['Recovered'].tolist()
predictions_s['Days Passed'] = test['Days Passed'].tolist()
predictions_s = predictions_s[['Days Passed','Local', 'Confirmed Prediction', 'Confirmed Actual','Deaths Prediction', 'Deaths Actual','Recovered Prediction','Recovered Actual']]
predictions_s

,Days Passed,Local,Confirmed Prediction,Confirmed Actual,Deaths Prediction,Deaths Actual,Recovered Prediction,Recovered Actual
0,13.0,Nepal,33.0,1,-5.0,0,44.0,0
1,2.0,Queensland / Australia,-65.0,0,-335.0,0,-392.0,0
2,44.0,Sao Tome and Principe,81.0,0,40.0,0,42.0,0
3,35.0,Iraq,49.0,5,-4.0,0,12.0,0
4,60.0,Faroe Islands / Denmark,136.0,115,-13.0,0,86.0,0
...,...,...,...,...,...,...,...,...
255,96.0,Togo,231.0,98,11.0,6,97.0,62
256,11.0,Lithuania,-9.0,0,43.0,0,-55.0,0
257,43.0,Cambodia,111.0,1,25.0,0,36.0,1
258,19.0,Cote d'Ivoire,-2.0,0,15.0,0,3.0,0


In [21]:
with pd.ExcelWriter('results_svr_s.xlsx') as writer:
    covid_data.to_excel(writer)

Tratamentos dos dados para a sua visualização. Arredondamento de valores para inteiros, uma vez que para certas variáveis apenas tem significado valores inteiros.

Guardar os resultados em formato xlsx para melhor visualização

## Sigmoid with epsilon=0.2

In [33]:
regressor_s2 = make_pipeline(StandardScaler(), SVR(kernel='sigmoid', epsilon=0.2, cache_size = 500))
wrapper_s2 = MultiOutputRegressor(regressor_s2)

In [ ]:
wrapper_s2.fit(train[x_columns],train[y_columns])
predictions_s2 = wrapper_s2.predict(test[x_columns])

In [22]:
predictions_s2 = pd.DataFrame(data=predictions_s2,columns=['Confirmed Prediction','Deaths Prediction','Recovered Prediction'])
predictions_s2['Local'] = test['Local'].tolist()
predictions_s2['Confirmed Prediction'] = predictions_s2['Confirmed Prediction'].apply(np.ceil)
predictions_s2['Deaths Prediction'] = predictions_s2['Deaths Prediction'].apply(np.ceil)
predictions_s2['Recovered Prediction'] = predictions_s2['Recovered Prediction'].apply(np.ceil)
predictions_s2['Confirmed Actual'] = test['Confirmed'].tolist()
predictions_s2['Deaths Actual'] = test['Deaths'].tolist()
predictions_s2['Recovered Actual'] = test['Recovered'].tolist()
predictions_s2['Days Passed'] = test['Days Passed'].tolist()
predictions_s2 = predictions_s2[['Days Passed','Local', 'Confirmed Prediction', 'Confirmed Actual','Deaths Prediction', 'Deaths Actual','Recovered Prediction','Recovered Actual']]
predictions_s2

,Days Passed,Local,Confirmed Prediction,Confirmed Actual,Deaths Prediction,Deaths Actual,Recovered Prediction,Recovered Actual
0,77.0,Tanzania,12.0,25,2.0,1,3.0,5
1,61.0,Maldives,11.0,13,1.0,0,2.0,0
2,102.0,Ethiopia,24.0,135,10.0,3,12.0,75
3,6.0,Antigua and Barbuda,10.0,0,-0.0,0,1.0,0
4,76.0,Portugal,608.0,12442,337.0,345,405.0,184
...,...,...,...,...,...,...,...,...
255,40.0,Angola,10.0,0,-0.0,0,1.0,0
256,7.0,Niger,10.0,0,-0.0,0,1.0,0
257,5.0,Spain,10.0,0,-0.0,0,1.0,0
258,62.0,Channel Islands / United Kingdom,13.0,36,3.0,0,3.0,0


In [23]:
with pd.ExcelWriter('results_svr_s2.xlsx') as writer:
    covid_data.to_excel(writer)

## Rbf com epsilon=0.2

In [22]:
regressor_rbf2 = make_pipeline(StandardScaler(), SVR(kernel='rbf', epsilon=0.2,cache_size=500))
wrapper_rbf2 = MultiOutputRegressor(regressor_rbf2)

In [ ]:
wrapper_rbf2.fit(train[x_columns],train[y_columns])
predictions_rbf2 = wrapper_rbf2.predict(test[x_columns])

In [ ]:
predictions_rbf2 = pd.DataFrame(data=predictions_rbf2,columns=['Confirmed Prediction','Deaths Prediction','Recovered Prediction'])
predictions_rbf2['Local'] = test['Local'].tolist()
predictions_rbf2['Confirmed Prediction'] = predictions_rbf2['Confirmed Prediction'].apply(np.ceil)
predictions_rbf2['Deaths Prediction'] = predictions_rbf2['Deaths Prediction'].apply(np.ceil)
predictions_rbf2['Recovered Prediction'] = predictions_rbf2['Recovered Prediction'].apply(np.ceil)
predictions_rbf2['Confirmed Actual'] = test['Confirmed'].tolist()
predictions_rbf2['Deaths Actual'] = test['Deaths'].tolist()
predictions_rbf2['Recovered Actual'] = test['Recovered'].tolist()
predictions_rbf2['Days Passed'] = test['Days Passed'].tolist()
predictions_rbf2 = predictions_rbf2[['Days Passed','Local', 'Confirmed Prediction', 'Confirmed Actual','Deaths Prediction', 'Deaths Actual','Recovered Prediction','Recovered Actual']]
predictions_rbf2

In [ ]:
with pd.ExcelWriter('results_svr_rbf2.xlsx') as writer:
    covid_data.to_excel(writer)